# Part One

In [12]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob
import copy 

### Pulling Info From YML File

In [13]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [14]:
def get_config_info(config_filename):
    with open(config_filename, "r") as yml_file:
        config = yaml.safe_load(yml_file)
    
    first_ID = config['inputs']['first_ID']
    second_ID = config['inputs']['second_ID']
    output_path = config['inputs']['output_path']
    first_ID_datafiles = config[first_ID].values()
    second_ID_datafiles = config[second_ID].values()
    
    return first_ID, second_ID, output_path, first_ID_datafiles, second_ID_datafiles

In [15]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [16]:
first_ID = config['inputs']['first_ID']

In [17]:
second_ID = config['inputs']['second_ID']

In [18]:
output_path = config['inputs']['output_path']

In [19]:
first_ID_datafiles = config[first_ID].values()

In [20]:
second_ID_datafiles = config[second_ID].values()

### Downloading SQL Data

In [21]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [22]:
# NOTE: Format to make it select correct MGRA series based on input DS
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [23]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [24]:
# SQl Data at different levels
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction'].drop('geotype', axis=1).drop_duplicates()
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa'].drop('geotype', axis=1).drop_duplicates()

### Downloading the first_ID Data

In [25]:
# maybe config argument?
def download_DS_data(ds_ID, jur_level):
    datafiles = config[ds_ID].values()
    
    df = pd.DataFrame()
    for file_name in datafiles:
        working_df = pd.read_csv(file_name)
        working_df['year'] = f"{file_name[-11:-7]}"
        df = df.append(working_df)
        
    # Save the features_first_ID for future use (Used when creating the diff file)
    features = df.drop(['mgra', 'year'], axis=1).columns
    
    comparison_no_geozone = copy.deepcopy(df)
    
    # Adding SQl Data to first_id_df
    comparison_processed_data = comparison_no_geozone.merge(jur_level, how='left', on='mgra')
    
    # making it original
    comparison_processed_data.columns = [x + f'_{ds_ID}' for x in comparison_processed_data.columns]
    
    #comparison_first_ID_processed_data.to_csv(f"{output_path}/comparison_{first_ID}_processed_data.csv")
    
    return comparison_processed_data, features

In [26]:
first_ID_temp, first_ID_features = download_DS_data(first_ID, jur_level)

In [27]:
first_ID_temp

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,hotelroomtotal_DS35,luz_id_DS35,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,geozone_DS35
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
2,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego
3,4,3358,30,30,0,0,30,30,0,0,...,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016,San Diego
4,5,3358,28,28,0,0,28,28,0,0,...,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016,San Diego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330299,22998,1290,92,92,0,0,87,87,0,0,...,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050,Carlsbad
330300,22999,1290,0,0,0,0,0,0,0,0,...,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050,Carlsbad
330301,23000,1290,131,131,0,0,126,126,0,0,...,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050,Carlsbad
330302,23001,1254,85,85,0,0,82,82,0,0,...,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050,Carlsbad


### Downloading  the second_ID Data

In [28]:
second_ID_temp, second_ID_features = download_DS_data(second_ID, jur_level)

In [29]:
second_ID_temp

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,geozone_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,San Diego
1,2,3331,35,35,0,0,34,34,0,0,...,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,San Diego
2,3,3358,52,52,0,0,52,52,0,0,...,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,San Diego
3,4,3358,30,30,0,0,30,30,0,0,...,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016,San Diego
4,5,3358,28,28,0,0,28,28,0,0,...,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016,San Diego
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
330299,22998,1290,92,92,0,0,87,87,0,0,...,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050,Carlsbad
330300,22999,1290,0,0,0,0,0,0,0,0,...,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050,Carlsbad
330301,23000,1290,131,131,0,0,126,126,0,0,...,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050,Carlsbad
330302,23001,1254,85,85,0,0,81,81,0,0,...,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050,Carlsbad


### Concatenate both DS dataframes

In [36]:
def concat_dfs(comparison_first_ID_processed_data, comparison_second_ID_processed_data):
    # Added geozone to merge keys to account for mgra's in multiple jurisdictions (or other geographical levels)
    first_second_ID_comparison = comparison_first_ID_processed_data.merge(
        comparison_second_ID_processed_data,
        how='left',
        left_on=[f'mgra_{first_ID}',
                 f'year_{first_ID}',
                 f'geozone_{first_ID}'],
        right_on=[f'mgra_{second_ID}',
                 f'year_{second_ID}',
                 f'geozone_{second_ID}'])
    
    # Clean green combined
    first_second_ID_comparison = first_second_ID_comparison.drop([f'mgra_{second_ID}', f'year_{second_ID}', f'geozone_{second_ID}'], axis=1)
    first_second_ID_comparison = first_second_ID_comparison.rename(columns={f'mgra_{first_ID}': 'mgra', f'year_{first_ID}': 'year', f'geozone_{first_ID}': 'geozone'})
    
    # Because we're summing, if using series 14 data, mgra's in multiple jurisdictions will be counted multiple times
    first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()
    
    #first_second_ID_comparison.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison")
    
    return first_second_ID_comparison

In [37]:
combined = concat_dfs(first_ID_temp, second_ID_temp)

### Creating the complete Diff File

In [43]:
def create_diff(features_first_ID, features_second_ID, first_second_ID_comparison):
    
    # Finding features common to both DSID data frames
    first_ID_unique = set(features_first_ID)
    intersection = first_ID_unique.intersection(features_second_ID)
    shared_features = list(intersection)

    # In case we want to display non-shared features
    non_shared_features = list(first_ID_unique ^ set(features_second_ID))
    
    # Calculate diff values between the two DS_ID's
    diff_df = pd.DataFrame()

    # NOTE: Subtracts second DS ID from first DS ID. If negative, then second DS ID was greater than first DS ID.
    for column in shared_features:
        diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']
    
    #diff_df.to_csv(f"{output_path}/{first_ID}_{second_ID}_complete_diff")
    
    return diff_df

In [44]:
temp_diff = create_diff(first_ID_features, second_ID_features, combined)

## CPA level Data

In [ ]:
def geo_level(combined_df):
    

In [36]:
# Adding SQl Data (CPA) to first_id_df
comparison_first_ID_processed_data_cpa = first_ID_df.merge(cpa_level, how='left', on='mgra')
comparison_first_ID_processed_data_cpa = comparison_first_ID_processed_data_cpa[comparison_first_ID_processed_data_cpa['geozone'] != '*Not in a CPA*']

In [37]:
# Adding SQl Data (CPA) to second_id_df
comparison_second_ID_processed_data_cpa = second_ID_df.merge(cpa_level, how='left', on='mgra')
comparison_second_ID_processed_data_cpa = comparison_second_ID_processed_data_cpa[comparison_second_ID_processed_data_cpa['geozone'] != '*Not in a CPA*']

In [38]:
# Merge first_id_df and second_id_df together on mgra, year, and geozone
comparison_processed_data_cpa = comparison_first_ID_processed_data_cpa.merge(comparison_second_ID_processed_data_cpa, how='inner', on=['mgra', 'year', 'geozone'], suffixes=[f'_{first_ID}', f'_{second_ID}'])

In [39]:
# Drop the MGRA column because it isn't really a quantitative value
comparison_processed_data_cpa = comparison_processed_data_cpa.drop('mgra', axis=1)

In [40]:
# Aggregate the sum of features by geozone and year
comparison_processed_data_cpa = comparison_processed_data_cpa.groupby(['geozone', 'year']).sum()

In [41]:
# Rename index (geozone -> cpa)
comparison_processed_data_cpa.index.names = ['cpa', 'year']

In [162]:
comparison_processed_data_cpa#.equals(temp_cpa)

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  \
cpa                       year                                              
32nd Street Naval Station 2016     86368      161          60         101   
                          2018     86368      163          62         101   
                          2020     86368      163          62         101   
                          2023     86368      163          62         101   
                          2025     86368      163          62         101   
...                                  ...      ...         ...         ...   
Via De La Valle           2032     55759      499         383         116   
                          2035     55759      500         384         116   
                          2040     55759      508         392         116   
                          2045     55759      508         392         116   
                          2050     55759      509         393         116   

                                hs_mh_DS35  hh_DS35  hh_sf_DS35  hh_mf_DS35  \
cpa                       year                                                
32nd Street Naval Station 2016           0      161          60         101   
                          2018           0      162          61         101   
                          2020           0      162          61         101   
                          2023           0      162          61         101   
                          2025           0      162          61         101   
...                                    ...      ...         ...         ...   
Via De La Valle           2032           0      462         353         109   
                          2035           0      462         353         109   
                          2040           0      464         355         109   
                          2045           0      464         355         109   
                          2050           0      465         356         109   

                                hh_mh_DS35  gq_civ_DS35  ...  \
cpa                       year                           ...   
32nd Street Naval Station 2016           0            0  ...   
                          2018           0            0  ...   
                          2020           0            0  ...   
                          2023           0            0  ...   
                          2025           0            0  ...   
...                                    ...          ...  ...   
Via De La Valle           2032           0            0  ...   
                          2035           0            0  ...   
                          2040           0            0  ...   
                          2045           0            0  ...   
                          2050           0            0  ...   

                                midpriceroom_DS41  upscaleroom_DS41  \
cpa                       year                                        
32nd Street Naval Station 2016                  0                 0   
                          2018                  0                 0   
                          2020                  0                 0   
                          2023                  0                 0   
                          2025                  0                 0   
...                                           ...               ...   
Via De La Valle           2032                  0                 0   
                          2035                  0                 0   
                          2040                  0                 0   
                          2045                  0                 0   
                          2050                  0                 0   

                                hotelroomtotal_DS41  luz_id_DS41  \
cpa                       year                                     
32nd Street Naval Station 2016                   25         2538   
                          2018           

In [43]:
#comparison_processed_data_cpa.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_cpa")

## Jurisdiction level Data

In [44]:
# Adding SQl Data (Jurisdiction) to first_id_df
comparison_first_ID_processed_data_jur = first_ID_df.merge(jur_level, how='left', on='mgra')

In [45]:
# Adding SQl Data (Jurisdiction) to second_id_df
comparison_second_ID_processed_data_jur = second_ID_df.merge(jur_level, how='left', on='mgra')

In [46]:
# Merge first_id_df and second_id_df together on mgra, year, and geozone
comparison_processed_data_jur = comparison_first_ID_processed_data_jur.merge(comparison_second_ID_processed_data_jur, how='inner', on=['mgra', 'year', 'geozone'], suffixes=[f'_{first_ID}', f'_{second_ID}'])

In [47]:
# Drop the MGRA column because it isn't really a quantitative value
comparison_processed_data_jur = comparison_processed_data_jur.drop('mgra', axis=1)

In [48]:
# Aggregate the sum of features by geozone and year
comparison_processed_data_jur = comparison_processed_data_jur.groupby(['geozone', 'year']).sum()

In [49]:
# Rename index (geozone -> jurisdiction)
comparison_processed_data_jur.index.names = ['jurisdiction', 'year']

In [50]:
comparison_processed_data_jur.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  \
jurisdiction year                                                          
Carlsbad     2016    870633    50670       34815       14215        1640   
             2018    870633    51501       35865       13994        1642   
             2020    870633    52666       36426       14598        1642   
             2023    870633    53363       36634       15087        1642   
             2025    870633    53899       36987       15270        1642   

                   hh_DS35  hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  \
jurisdiction year                                                             
Carlsbad     2016    47670       32817       13351        1502          915   
             2018    48718       33985       13229        1504          915   
             2020    49783       34559       13719        1505          915   
             2023    50383       34761       14117        1505          915   
             2025    50868       35050       14313        1505          915   

                   ...  midpriceroom_DS41  upscaleroom_DS41  \
jurisdiction year  ...                                        
Carlsbad     2016  ...                940               765   
             2018  ...               1026               837   
             2020  ...               1026               837   
             2023  ...               1056               862   
             2025  ...               1076               879   

                   hotelroomtotal_DS41  luz_id_DS41  truckregiontype_DS41  \
jurisdiction year                                                           
Carlsbad     2016                 3406        21520                   811   
             2018                 3722        21520                   811   
             2020                 3722        21520                   811   
             2023                 3833        21520                   811   
             2025                 3905        21520                   811   

                   district27_DS41  milestocoast_DS41    acres_DS41  \
jurisdiction year                                                     
Carlsbad     2016              943          1918.0907  27460.971113   
             2018              943          1918.0907  27460.971113   
             2020              943          1918.0907  27460.971113   
             2023              943          1918.0907  27460.971113   
             2025              943          1918.0907  27460.971113   

                   effective_acres_DS41  land_acres_DS41  
jurisdiction year                                         
Carlsbad     2016          16381.889663     26574.857225  
             2018          16381.889663     26574.857225  
             2020          16381.889663     26574.857225  
             2023          16381.889663     26574.857225  
             2025          16381.889663     26574.857225  

[5 rows x 206 columns]

In [51]:
#comparison_processed_data_jur.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_jurisdiction")

## Region level Data

In [52]:
# Merge first_id_df and second_id_df together on mgra and year
comparison_processed_data_reg = first_ID_df.merge(second_ID_df, how='inner', on=['mgra', 'year'], suffixes=[f'_{first_ID}', f'_{second_ID}'])

In [53]:
# Aggregate the sum of features by year
comparison_processed_data_reg = comparison_processed_data_reg.groupby('year').sum()

In [54]:
# Drop the MGRA column because it isn't really a quantitative value
comparison_processed_data_reg = comparison_processed_data_reg.drop('mgra', axis=1)

In [55]:
comparison_processed_data_reg.head()

,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,gq_civ_DS35,...,midpriceroom_DS41,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41
year,,,,,,,,,,,,,,,,,,,,,
2016,53370775,1190554,717626,430716,42212,1134848,687509,407158,40181,63014,...,15396,13062,56646,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2018,53370775,1205852,723209,440536,42107,1147635,692426,415193,40016,69372,...,16823,14280,61917,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2020,53370775,1226462,730452,453903,42107,1166240,698993,427158,40089,72056,...,16823,14280,61917,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2023,53370775,1262957,739359,481491,42107,1197072,705785,451128,40159,73875,...,17308,14694,63707,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06
2025,53370775,1288217,746460,499650,42107,1219745,711877,467676,40192,74447,...,17627,14970,64889,2431123,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06


In [56]:
#comparison_processed_data_reg.to_csv(f"{output_path}/{first_ID}_{second_ID}_comparison_region")

# Modularizing Part 1 Functions

# Brainstorming for Part 2

In [57]:
config['dof']['dof_data'] # figure out what this is for and what to do with it

'C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-01 Regional Forecast DS 41-42/dof_data.xlsx'

In [58]:
first_second_ID_comparison.head()

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra year                                                                   
1    2016      3331       19          19           0           0       18   
     2018      3331       19          19           0           0       18   
     2020      3331       19          19           0           0       18   
     2023      3331       20          20           0           0       18   
     2025      3331       20          20           0           0       18   

           hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra year                                                   ...   
1    2016          18           0           0            0  ...   
     2018          18           0           0            0  ...   
     2020          18           0           0            0  ...   
     2023          18           0           0            0  ...   
     2025          18           0           0            0  ...   

           midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra year                                                             
1    2016                  0                 0                    0   
     2018                  0                 0                    0   
     2020                  0                 0                    0   
     2023                  0                 0                    0   
     2025                  0                 0                    0   

           luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra year                                                       
1    2016           95                     1               27   
     2018           95                     1               27   
     2020           95                     1               27   
     2023           95                     1               27   
     2025           95                     1               27   

           milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra year                                                        
1    2016             3.7997   16.615444             12.961482   
     2018             3.7997   16.615444             12.961482   
     2020             3.7997   16.615444             12.961482   
     2023             3.7997   16.615444             12.961482   
     2025             3.7997   16.615444             12.961482   

           land_acres_DS41  
mgra year                   
1    2016        16.615444  
     2018        16.615444  
     2020        16.615444  
     2023        16.615444  
     2025        16.615444  

[5 rows x 206 columns]

In [59]:
# MGRA level stats
first_second_ID_comparison.describe(percentiles=[0.5])

,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,gq_civ_DS35,...,midpriceroom_DS41,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41
count,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,...,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000,299026.000000
mean,2543.990305,63.173637,36.615990,24.473999,2.083648,59.638944,34.869941,22.776053,1.992950,3.359902,...,0.828159,0.665678,2.947510,116.483871,1.104600,10.250804,11.668477,127.805072,60.829469,126.176969
std,1752.794996,129.972638,82.686039,89.589628,26.589254,123.192874,80.436394,83.229181,25.528879,75.535876,...,15.606058,15.896093,36.418048,80.692463,0.321013,7.747927,11.714641,1109.942646,750.828075,1107.322257
min,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.010300,0.225724,0.000000,0.000000
50%,2412.000000,30.000000,16.000000,0.000000,0.000000,28.000000,15.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,106.000000,1.000000,10.000000,7.996850,18.309491,15.628349,18.218451
max,14457.000000,11622.000000,8288.000000,4000.000000,2073.000000,11146.000000,8230.000000,3095.000000,2010.000000,5496.000000,...,931.000000,1254.000000,2136.000000,684.000000,3.000000,52.000000,70.450400,99278.194490,99094.679110,99136.666380


### Outlier Detection

In [81]:
# Good article: https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/

In [63]:
diff_stats = diff_df.describe()

In [66]:
diff_stats

,hh_mh_diff,luxuryroom_diff,emp_whsle_whs_diff,district27_diff,dudenbin_diff,emp_total_diff,economyroom_diff,emp_state_local_gov_blue_diff,emp_st_lcl_gov_accts_diff,pop_diff,...,emp_personal_svcs_office_diff,duden_diff,parkactive_diff,hs_mf_diff,parkarea_diff,i2_diff,i6_diff,mstallsoth_diff,i5_diff,retempden_diff
count,299026.000000,299026.0,299026.000000,299026.0,299026.0,299026.000000,299026.0,299026.000000,299026.0,299026.000000,...,299026.000000,2.990260e+05,299026.0,299026.0,299026.0,299026.000000,299026.000000,299026.0,299026.000000,2.990260e+05
mean,0.000057,0.0,-0.000615,0.0,0.0,-0.001401,0.0,-0.000023,0.0,-0.012909,...,-0.004572,-2.987001e-25,0.0,0.0,0.0,0.000696,0.001515,0.0,0.001645,9.046036e-27
std,0.048279,0.0,1.414147,0.0,0.0,76.044392,0.0,2.937020,0.0,18.795366,...,1.508903,1.627565e-22,0.0,0.0,0.0,3.398388,3.747223,0.0,3.263008,1.945374e-23
min,-5.000000,0.0,-97.000000,0.0,0.0,-8382.000000,0.0,-253.000000,0.0,-1662.000000,...,-336.000000,-8.900000e-20,0.0,0.0,0.0,-78.000000,-112.000000,0.0,-57.000000,-4.720000e-21
25%,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,-4.000000,...,0.000000,0.000000e+00,0.0,0.0,0.0,-1.000000,-1.000000,0.0,-1.000000,0.000000e+00
50%,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000e+00
75%,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,4.000000,...,0.000000,0.000000e+00,0.0,0.0,0.0,1.000000,1.000000,0.0,1.000000,0.000000e+00
max,3.000000,0.0,140.000000,0.0,0.0,3267.000000,0.0,253.000000,0.0,610.000000,...,60.000000,0.000000e+00,0.0,0.0,0.0,69.000000,52.000000,0.0,86.000000,9.446000e-21


In [70]:
from scipy import stats

In [73]:
first_second_ID_comparison

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra  year                                                                   
1     2016      3331       19          19           0           0       18   
      2018      3331       19          19           0           0       18   
      2020      3331       19          19           0           0       18   
      2023      3331       20          20           0           0       18   
      2025      3331       20          20           0           0       18   
...              ...      ...         ...         ...         ...      ...   
23002 2032      1254      120          20         100           0      100   
      2035      1254      120          20         100           0      100   
      2040      1254      120          20         100           0      106   
      2045      1254      120          20         100           0      107   
      2050      1254      120          20         100           0      109   

            hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra  year                                                   ...   
1     2016          18           0           0            0  ...   
      2018          18           0           0            0  ...   
      2020          18           0           0            0  ...   
      2023          18           0           0            0  ...   
      2025          18           0           0            0  ...   
...                ...         ...         ...          ...  ...   
23002 2032          14          86           0            0  ...   
      2035          14          86           0            0  ...   
      2040          16          90           0            0  ...   
      2045          17          90           0            0  ...   
      2050          17          92           0            0  ...   

            midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra  year                                                             
1     2016                  0                 0                    0   
      2018                  0                 0                    0   
      2020                  0                 0                    0   
      2023                  0                 0                    0   
      2025                  0                 0                    0   
...                       ...               ...                  ...   
23002 2032                  0                 0                    0   
      2035                  0                 0                    0   
      2040                  0                 0                    0   
      2045                  0                 0                    0   
      2050                  0                 0                    0   

            luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra  year                                                       
1     2016           95                     1               27   
      2018           95                     1               27   
      2020           95                     1               27   
      2023           95                     1               27   
      2025           95                     1               27   
...                 ...                   ...              ...   
23002 2032           14                     1                1   
      2035           14                     1                1   
      2040           14                     1                1   
      2045           14                     1                1   
      2050           14                     1                1   

            milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra  year                                                        
1     2016             3.7997   16.615444             12.961482   
      2018             3.7997   16.615444             12.961482   
      2020             3.7997   16.615444       

In [80]:
q = first_second_ID_comparison["taz_DS35"].quantile(0.99)
first_second_ID_comparison[first_second_ID_comparison["taz_DS35"] > q]

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra  year                                                                   
1387  2016     11976      123         123           0           0      114   
      2018     11976      123         123           0           0      114   
      2020     11976      123         123           0           0      114   
      2023     11976      126         126           0           0      114   
      2025     11976      129         129           0           0      117   
...              ...      ...         ...         ...         ...      ...   
22843 2032      9490        0           0           0           0        0   
      2035      9490        0           0           0           0        0   
      2040      9490        0           0           0           0        0   
      2045      9490        0           0           0           0        0   
      2050      9490        0           0           0           0        0   

            hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra  year                                                   ...   
1387  2016         114           0           0            0  ...   
      2018         114           0           0            0  ...   
      2020         114           0           0            0  ...   
      2023         114           0           0            0  ...   
      2025         117           0           0            0  ...   
...                ...         ...         ...          ...  ...   
22843 2032           0           0           0            0  ...   
      2035           0           0           0            0  ...   
      2040           0           0           0            0  ...   
      2045           0           0           0            0  ...   
      2050           0           0           0            0  ...   

            midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra  year                                                             
1387  2016                  0                 0                    0   
      2018                  0                 0                    0   
      2020                  0                 0                    0   
      2023                  0                 0                    0   
      2025                  0                 0                    0   
...                       ...               ...                  ...   
22843 2032                  0                 0                    0   
      2035                  0                 0                    0   
      2040                  0                 0                    0   
      2045                  0                 0                    0   
      2050                  0                 0                    0   

            luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra  year                                                       
1387  2016          330                     3               30   
      2018          330                     3               30   
      2020          330                     3               30   
      2023          330                     3               30   
      2025          330                     3               30   
...                 ...                   ...              ...   
22843 2032          250                     2               22   
      2035          250                     2               22   
      2040          250                     2               22   
      2045          250                     2               22   
      2050          250                     2               22   

            milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra  year                                                        
1387  2016            26.5146   34.608571             34.608571   
      2018            26.5146   34.608571             34.608571   
      2020            26.5146   34.608571       

In [ ]:
first_second_ID_comparison[first_second_ID_comparison["taz_DS35"] > q]

In [78]:
first_second_ID_comparison['taz_DS35']

mgra   year
1      2016    3331
       2018    3331
       2020    3331
       2023    3331
       2025    3331
               ... 
23002  2032    1254
       2035    1254
       2040    1254
       2045    1254
       2050    1254
Name: taz_DS35, Length: 299026, dtype: int64

In [72]:
first_second_ID_comparison[(np.abs(stats.zscore(first_second_ID_comparison)) < 3).all(axis=1)]

,,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,gq_civ_DS35,...,midpriceroom_DS41,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41
mgra,year,,,,,,,,,,,,,,,,,,,,,


In [ ]:
diff_stats.loc['max']

In [65]:
diff_stats.loc['max']

hh_mh_diff            3.000000e+00
luxuryroom_diff       0.000000e+00
emp_whsle_whs_diff    1.400000e+02
district27_diff       0.000000e+00
dudenbin_diff         0.000000e+00
                          ...     
i2_diff               6.900000e+01
i6_diff               5.200000e+01
mstallsoth_diff       0.000000e+00
i5_diff               8.600000e+01
retempden_diff        9.446000e-21
Name: max, Length: 103, dtype: float64